In [1]:
import sys
import os
path = os.path.abspath('../..')
if path not in sys.path:
  sys.path.insert(0, path)

%load_ext autoreload

In [2]:
import collections
import concurrent.futures
import datetime
import json
import termcolor
import re

import numpy as np

from IPython import display

from concordia.agents import basic_agent
from concordia.components import agent as components
from concordia import components as generic_components
from concordia.associative_memory import associative_memory
from concordia.associative_memory import blank_memories
from concordia.associative_memory import formative_memories
from concordia.associative_memory import importance_function
from concordia.clocks import game_clock
from concordia.components import game_master as gm_components
from concordia.environment import game_master
from concordia.metrics import goal_achievement
from concordia.metrics import common_sense_morality
from concordia.metrics import opinion_of_others
from concordia.utils import html as html_lib
from concordia.utils import measurements as measurements_lib
from concordia.utils import plotting

from examples.custom_components import behavior

In [3]:
local_models = True

if local_models:
    # Setup LLM
    from concordia.language_model import ollama_model
    model = ollama_model.OllamaLanguageModel(
        model_name='gemma',
        streaming=True
    )

    # Setup measurements and clock
    measurements = measurements_lib.Measurements()
    time_step = datetime.timedelta(minutes=20)
    SETUP_TIME = datetime.datetime(hour=20, year=2024, month=10, day=1)

    START_TIME = datetime.datetime(hour=18, year=2024, month=10, day=2)
    clock = game_clock.MultiIntervalClock(
        start=SETUP_TIME,
        step_sizes=[time_step, datetime.timedelta(seconds=10)])

    # Setup sentence encoder
    from sentence_transformers import SentenceTransformer
    st5_model = SentenceTransformer('sentence-transformers/sentence-t5-base')
    embedder = st5_model.encode

else:
    from concordia.language_model import gpt_model
    from concordia.language_model import gcloud_model
    # @title Language Model - pick your model and provide keys
    CLOUD_PROJECT_ID = '' #@param {type: 'string'}
    GPT_API_KEY = '' #@param {type: 'string'}
    GPT_MODEL_NAME = '' #@param {type: 'string'}

    USE_CLOUD = True #@param {type: 'boolean'}

    if USE_CLOUD:
        model = gcloud_model.CloudLanguageModel(project_id= CLOUD_PROJECT_ID)
    else:
        model = gpt_model.GptLanguageModel(api_key=GPT_API_KEY, model_name=GPT_MODEL_NAME)

#@title Importance models
importance_model = importance_function.ConstantImportanceModel()
importance_model_gm = importance_function.ConstantImportanceModel()


In [4]:
# @title Generic memories are memories that all players and GM share.

shared_memories = [
    'Alice returns late from work and finds Dorothy at her doorstep.',
    "Dorothy has to move out of her house tonight and needs someone to help her move.",
]

# The generic context will be used for the NPC context. It reflects general
# knowledge and is possessed by all characters.
shared_context = model.sample_text(
    'Summarize the following passage in a concise and insightful fashion:\n'
    + '\n'.join(shared_memories)
    + '\n'
    + 'Summary:'
)

/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Sure, here is a concise summary of the passage in a concise and insightful fashion:

Alice returns home late from work and finds Dorothy at her doorstep, needing assistance with her move. Due to an unforeseen circumstance, Dorothy must relocate and requires help moving her belongings.Sure, here is a concise summary of the passage in a concise and insightful fashion:

Alice returns home late from work and finds Dorothy at her doorstep, needing assistance with her move. Due to an unforeseen circumstance, Dorothy must relocate and requires help moving her belongings.


In [5]:
blank_memory_factory = blank_memories.MemoryFactory(
    model=model,
    embedder=embedder,
    importance=importance_model.importance,
    clock_now=clock.now,
)

formative_memory_factory = formative_memories.FormativeMemoryFactory(
    model=model,
    shared_memories=shared_memories,
    blank_memory_factory_call=blank_memory_factory.make_blank_memory,
)

In [6]:
#@title Creating character backgrounds, goals and traits. Modify to explore how it influences the outcomes
NUM_PLAYERS = 2

scenario_premise = [

    (
        'Alice returns late from work and finds her friend Dorothy at her doorstep. '
        + 'Dorothy has to move out of her house tonight and needs someone to help her move.'
    ),
]
player_configs = [
    formative_memories.AgentConfig(
        name='Alice',
        gender='female',
        goal='Alice is exhausted after work, and she needs to help her grandmother all day the next day.',
        context=shared_context,
        traits='responsibility: medium; aggression: medium',
    ),
    formative_memories.AgentConfig(
        name='Dorothy',
        gender='female',
        goal=(
            "Dorothy has planned on Bob helping her move, but Bob didn't show up. "
        ),
        context=shared_context,
        traits='responsibility: medium; aggression: medium',
    ),
]

In [7]:
mem = formative_memory_factory.make_memories(player_configs[0])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

## Alice: A Woman of Many Talents and Complex Dreams

**Job:** Graphic Designer

**Typical Day:**

* Wake up and start early, usually around 8:00 AM.
* Work on freelance projects and client commissions.
* Design logos, marketing materials, and other visual content.
* Collaborate with clients and team members.
* Wrap up projects and prepare for the next day.
* Usually finishes work by 6:00 PM.

**Goals:**

* To establish herself as a leading designer in the industry.
* To travel the world and experience different cultures.
* To create impactful visual designs that bring joy to others.
* To build a successful career and achieve financial stability.

**Desires:**

* To have a creative outlet that allows her to express herself freely.
* To be recognized for her talent and creativity.
* To have a comfortable and fulfilling life.
* To find love and companionship.

**Hopes and Dreams:**

* To have her own design firm and work on high-profile projects.
* To travel the world and design landmark

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

## Alice's Formative Episode at Age 3

Three-year-old Alice was returning home late from work one evening when she found her neighbor, Dorothy, at her doorstep. Dorothy had to relocate due to an unforeseen circumstance and needed Alice's help moving her belongings. With determination and a sprinkle of creativity, Alice rallied her friends to assist her in moving Dorothy's possessions, transforming the moving process into a collaborative and joyful experience.

## Alice's Formative Episode at Age 7

Seven-year-old Alice was returning home late from work one evening when she found her neighbor, Dorothy, at her doorstep. Dorothy had to relocate due to an unforeseen circumstance and required help moving her belongings. With her characteristic creativity and resourcefulness, Alice devised a plan to move Dorothy's possessions using cardboard boxes and her friends' help. The moving process became a collaborative endeavor filled with laughter and joy.

...

## Alice's Formative Episode at Age 21

Twenty-one-year-old Alice was returning home late from work one evening when she found her neighbor, Dorothy, at her doorstep. Dorothy had to relocate due to an unforeseen circumstance and required help moving her belongings. With a mix of determination and creativity, Alice devised a plan to move Dorothy's possessions using her friends' help and repurposed cardboard boxes. The moving process became a collaborative endeavor filled with laughter and joy.

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
attitude_ratings = model.sample_text(
  f'Memories of Alice:{mem.retrieve_recent(100)}'
  f'Instructions: \n'
  f'Given the memories above, generate a list of five potential behaviours that alice can take in response to the situation. '
  f'For each potential behaviour, provide three possible positive and three possible negative consequences of that behaviour. '
  f'For each potential consequence, indicate with a number from -10 to 10 how bad to good '
  f'that consequence is for Alice. -10 is the worst possible outcome, 0 is neutral, and 10 is the best possible outcome.'
  f'List the positive or negative value after each of the separate consequences to each behaviour. '
  f'For each potential behaviour, provide just the consequence numbers separated by a comma. '
  f'For each potential consequence, indicate with a number from 0 to 100 how likely that consequence is to occur. 0 is impossible, 100 is certain. '
  f'For each behaviour, a set of possible consequences are now given. Not all consequences are likely to occur if a behaviour is done. '
  f'List the likelihood of each consequences if the behaviour is done after each of the separate consequences to each behaviour. '
  f'This is not the likelihood that Alice will take the action, but rather the likelihood that the consequence will occur if the action is done. '
  f'List the likelihood of the consequences after each of the separate consequences to each behaviour. So, '
  f'each of the 6 potential separate consequences should have a likelihood value. '
  f'This should be in the form of (Value: number, Likelihood: number) for each potential consequence, '
  f'Remember, there should be three separate positive and three negative consequences for each potential behaviour '
  f'each with its own value and likelihood.\n'
  f'Double check that you did all of the behaviours and people correctly, for example that the numbers are all provided.'
)

## Alice's Potential Behaviours and Consequences

**Potential Behaviour 1: Help Dorothy move her belongings**

* **Positive Consequences:**
    1. Gain respect and admiration from Dorothy and friends. (Value: 8, Likelihood: 80%)
    2. Sense of accomplishment and satisfaction. (Value: 7, Likelihood: 70%)
    3. Opportunity to demonstrate leadership and creativity. (Value: 9, Likelihood: 60%)

* **Negative Consequences:**
    1. Physical exhaustion and stress. (Value: -2, Likelihood: 40%)
    2. Damage to Dorothy's belongings. (Value: -5, Likelihood: 30%)
    3. Conflict with friends due to workload. (Value: -4, Likelihood: 20%)

**Potential Behaviour 2: Delegate the task to others**

* **Positive Consequences:**
    1. Gain more free time for personal activities. (Value: 6, Likelihood: 75%)
    2. Reduce physical exhaustion and stress. (Value: 8, Likelihood: 65%)
    3. Allow others to experience the sense of accomplishment. (Value: 7, Likelihood: 55%)

* **Negative Consequences:**
   

In [15]:
json_output = model.sample_text(
  f"Given the following list of behaviours and potential consequences:\n"
  f"{attitude_ratings}"
  f"\n"
  f"Convert the list into an array of JSON objects. "
  f"Include the following entries for each JSON object: \n"
  f"'behaviour': The behaviour\n"
  f"'consequences': An array of JSON objects including a description of the consequence, "
  f"the value of the consequence and the likelihood of the consequence.",
  max_tokens = 5000,
  max_characters = 5000,
  terminators=None
)

```json
[
  {
    "behaviour": "Help Dorothy move her belongings",
    "consequences": [
      {
        "description": "Gain respect and admiration from Dorothy and friends",
        "value": 8,
        "likelihood": 80
      },
      {
        "description": "Sense of accomplishment and satisfaction",
        "value": 7,
        "likelihood": 70
      },
      {
        "description": "Opportunity to demonstrate leadership and creativity",
        "value": 9,
        "likelihood": 60
      },
      {
        "description": "Physical exhaustion and stress",
        "value": -2,
        "likelihood": 40
      },
      {
        "description": "Damage to Dorothy's belongings",
        "value": -5,
        "likelihood": 30
      },
      {
        "description": "Conflict with friends due to workload",
        "value": -4,
        "likelihood": 20
      }
    ]
  },
  {
    "behaviour": "Delegate the task to others",
    "consequences": [
      {
        "description": "Gain more free ti

In [21]:
import json

json.loads(json_output.replace('```json', '').replace('```', '').replace(',\n  ...', '').strip())

[{'behaviour': 'Help Dorothy move her belongings',
  'consequences': [{'description': 'Gain respect and admiration from Dorothy and friends',
    'value': 8,
    'likelihood': 80},
   {'description': 'Sense of accomplishment and satisfaction',
    'value': 7,
    'likelihood': 70},
   {'description': 'Opportunity to demonstrate leadership and creativity',
    'value': 9,
    'likelihood': 60},
   {'description': 'Physical exhaustion and stress',
    'value': -2,
    'likelihood': 40},
   {'description': "Damage to Dorothy's belongings",
    'value': -5,
    'likelihood': 30},
   {'description': 'Conflict with friends due to workload',
    'value': -4,
    'likelihood': 20}]},
 {'behaviour': 'Delegate the task to others',
  'consequences': [{'description': 'Gain more free time for personal activities',
    'value': 6,
    'likelihood': 75},
   {'description': 'Reduce physical exhaustion and stress',
    'value': 8,
    'likelihood': 65},
   {'description': 'Allow others to experience th

In [11]:
def decompose(attitudes):

    attitude_list = re.split(
        r'\n[0-9]\.\s',
        attitudes
    )

    behaviour_list = []

    for attitude in attitude_list[1:]:
        
        behaviours = {}

        items = re.split(
            r'\.\n',
            attitude,
            maxsplit = 1
        )

        behaviours["behaviour"] = items[0]

        items = re.split(
            r'(?:Positive|Negative)\sconsequences:\n',
            items[1]
        )

        items = [item for item in items if item]

        behaviours["consequences"] = []

        for item in items:

            consequences = re.split(
                r'\*\s',
                item
            )
            consequences = [consequence.strip() for consequence in consequences if consequence]

            for consequence in consequences:

                description = re.match(
                    r'(.*)(?=\s\()',
                    consequence
                ).group(1)

                value = int(re.search(
                    r'(?<=Value:\s).*(?=,)',
                    consequence
                ).group(0))

                likelihood = int(re.search(
                    r'(?<=Likelihood:\s).*(?=\))',
                    consequence
                ).group(0))

                behaviours["consequences"].append({
                    "description": description,
                    "value": value,
                    "likelihood": likelihood
                })

        behaviour_list.append(behaviours)
    
    return behaviour_list

behaviour_list = decompose(attitude_ratings)

In [97]:
normative_ratings = model.sample_text(
    f'instructions: \n'
    f'below is a set out of potential behaviours that Alice can take in response to the situation. and how she feels about the personally. '
    f'find each of the behaviours (not the consequeces) and provide 5 people that might have opinions about whether Alice does this behaviour or not. ' 
    f'for each behaviour, indicate who the person is that is having the opinion, and give a rating from -10 to 10 whether they '
    f'approve or disapprove of the behaviour. -10 is the most disapproval,  and 10 is the most approval. '
    f'list just each potential behaviour and the 5 people who have opinions on it, and the rating of the person. ' +
    attitude_ratings
)

 
1. Helping Dorothy move immediately without hesitation.

* Sarah (Alice's best friend): 8 (very approving)
* Jack (Alice's brother): 6 (somewhat approving)
* Emily (Alice's colleague): 5 (neutral)
* Tim (Alice's partner): 4 (somewhat disapproving)
* Jessica (Dorothy's neighbor): -7 (very disapproving)

2. Asking Dorothy if she can help her move at a later time.

* Rachel (Alice's sister): 6 (somewhat approving)
* David (Alice's boss): 5 (neutral)
* Samantha (Alice's friend): 4 (somewhat disapproving)
* Tom (Dorothy's son): -8 (very disapproving)
* Lisa (Alice's acquaintance): -6 (somewhat disapproving)

3. Offering to help Dorothy move the next day, but only for a few hours.

* Michael (Alice's friend): 7 (approving)
* Karen (Alice's mother): 6 (somewhat approving)
* Olivia (Alice's coworker): 5 (neutral)
* Alex (Dorothy's grandson): -7 (very disapproving)
* Sophia (Alice's acquaintance): -6 (somewhat disapproving)

4. Suggesting that Dorothy hire professional movers instead.

* Amel

In [80]:
behav_string = "\n".join([f"{i+1}. {behaviour_list[i]['behaviour']}" for i in range(len(behaviour_list))])

motivation_ratings = model.sample_text(
  f'Memories of Alice: {behaviors._memory.retrieve_recent(100)}'
  f'Potential behaviours of Alice: '
  f'{behav_string}'
  f'Instructions: \n'
  f'List six people or groups of people that would be affected if Alice took any of the above behaviours. '
  f'For each person or group, provide a value between 0 to 100 indicating how much '
  f'Alice is motivated to take their approval or disapproval into account. '
  f'0 would mean that Alice does not care at all about their approval or disapproval, and 100 would mean that she cares the most. \n'
  f'Be sure to include numbers for each rating and allow for appropriate variation in the ratings. '
  f"Rating should be in the form of (Alice's motivation: number) following each person or group of people. "
  f"Double check that you provided a value for Alice's motivation to take each person or group of peopleinto account. "
  f'Provide this information as succinctly as possible. '
  f'Here is an example: \n'
  f"* Person 1 (Alice's motivation: 100) \n"
  f"* Person 2 (Alice's motivation: 0) \n"
  f"* Person 3 (Alice's motivation: 50) \n"
  f'etc.'
  f'Present all of the people consistently according to this format. Do not provide any explanations.'
)


* Dorothy (Alice's motivation: 100)
* Alice's mother (Alice's motivation: 80)
* Alice's friends (Alice's motivation: 50)
* Alice's colleagues (Alice's motivation: 40)
* Alice's client (Alice's motivation: 70)
* Alice's grandmother (Alice's motivation: 90)

In [88]:
def organize_people(ratings):

    rating_list = re.split(
        r'\*\s',
        ratings
    )
    # Alternative: it does it with numbers
    if len(rating_list) == 1:
        rating_list = re.split(
            r'[0-9]\.\s',
            ratings
        )
    
    people = []

    for rating in rating_list[1:]:
        name = re.match(
            r'(.*)(?=\s\()',
            rating
        ).group(1)
        motivation = re.search(
            r'(?<=motivation:\s)(.*)(?=\))',
            rating
        ).group(1)
        people.append({
            "name": name,
            "motivation": int(motivation)
        })
    
    return people

relevant_people = organize_people(motivation_ratings)

In [93]:
people_string = "\n".join([f"{i+1}. {relevant_people[i]['name']}" for i in range(len(relevant_people))])

approval_ratings = model.sample_text(
  f'Memories of Alice: {behaviors._memory.retrieve_recent(100)}'
  f'Potential behaviours of Alice: '
  f'{behav_string}'
  f"People affected by Alice's potential behaviours: "
  f'{people_string}'
  f'Instructions: \n'
  f'For each of the potential behaviours that Alice could take in response to the situation, '
  f"as well as each of the people who could be affected by Alice's behaviour, "
  f'provide a rating from -10 to 10 about whether they would approve or disapprove of the behaviour '
  f'if Alice were to engage in the behaviour. -10 would be the least approval and 10 would be the most approval. '
  f'this should be in the form of (Approval: number) following each person. '
  f'Double check that you did all of the behaviours and people correctly.'
  f'for example, each of the behaviours should be listed, and each should have a number for approval '
  f'for each of the people or groups of people.'
  f'Provide this information as succinctly as possible. '
  f'Here is an example: \n'
  f'Behaviour 1: \n'
  f'* Person 1 (Approval: 5) \n'
  f'* Person 2 (Approval: 10) \n'
  f'* Person 3 (Approval: -5) \n'
  f'etc.'
  f'Present all of the people consistently according to this format. Do not provide any explanations.'
)


Here are the potential behaviors of Alice and their impact on the people affected:

Behavior 1: Helping Dorothy move immediately without hesitation

* Dorothy (Approval: 10)
* Alice's mother (Approval: 8)
* Alice's friends (Approval: 5)
* Alice's colleagues (Approval: 5)
* Alice's client (Approval: 5)
* Alice's grandmother (Approval: 10)

Behavior 2: Asking Dorothy if she can help another time

* Dorothy (Approval: 7)
* Alice's mother (Approval: 9)
* Alice's friends (Approval: 6)
* Alice's colleagues (Approval: 7)
* Alice's client (Approval: 8)
* Alice's grandmother (Approval: 9)

Behavior 3: Asking a friend or family member to help Dorothy move instead

* Dorothy (Approval: 6)
* Alice's mother (Approval: 7)
* Alice's friends (Approval: 8)
* Alice's colleagues (Approval: 7)
* Alice's client (Approval: 7)
* Alice's grandmother (Approval: 7)

Behavior 4: Telling Dorothy that she can't help with the move right now

* Dorothy (Approval: 3)
* Alice's mother (Approval: 5)
* Alice's friends 

In [ ]:
find_ratings = model.sample_text(
    f'given the file below, ' +
    attitude_ratings +
    f'instructions: \n'
    f'create a list of the values and likelihoods for each of the potential consequences. '
    f' for each behaviour found in ' 
    f'Memories of Alice:{behaviors._memory.retrieve_recent(100)}'
    f'provide only the numbers with the values in one list and the likelihoods in another list.'
    f'be sure to do for all behaviours and all potential consequences.'
    f'you should but this into a JSON array with the values and likelihoods as separate lists.'
    f'only the numbers should be in the JSON array.'
)

find_norms = model.sample_text(
    f'given the file below, ' +
    normative_ratings +
    f'instructions: \n'
    f'create a list of the approvals and motivates to comply for each of the behaviours. '
    f' for each behaviour found in ' 
    f'Memories of Alice:{behaviors._memory.retrieve_recent(100)}'
    f'provide only the approvals in one list and the motivations to comply in another list for each behaviour and each of the people under consideration.'
    f'be sure to do for all behaviours and all people that may care.'
    f'you should but this into a JSON array with the approvals and motivations as separate lists.'
    f'only the numbers should be in the JSON array.'
    f'have it be one long JSON array for each of the approvals and motivations rather than separated by behaviour.'
)